<h1 style="text-align:center">Basic Block Diagram Operations</h1>
<h3 style="text-align:center">MCHE474: Control Systems</h3> 
<p style="text-align:center">Dr. Joshua Vaughan <br>
<a href="mailto:joshua.vaughan@louisiana.edu">joshua.vaughan@louisiana.edu</a><br>
<a href="http://www.ucs.louisiana.edu/~jev9637/">http://www.ucs.louisiana.edu/~jev9637/</a></p>

In this notebook, we'll take a quick look at some of the  ways to manipulate transfer functions using the [Control System Library](http://python-control.readthedocs.io/en/latest/index.html), mimicing the block diagram algebra that we can do by hand. 

## Notebook Imports

AS usual, we'll start by importing the libraries we'll need to work.  These include [NumPy](http://www.numpy.org), [matplotlib](https://matplotlib.org) for plotting, and the Control Systems Library itself. The code cells here will appear unchanged in most of the notebooks we'll use in *MCHE474*.

We first import `numpy` and give it a "nickname" `np`. In doing so, we can preface calls to all NumPy functions with `np.` rather than having to type `numpy.`. This is a common convention in the use of NumPy. We say that we have imported NumPy into the namespace `np`. I'm oversimplifying a bit, but having different namespaces allows libraries to have functions of the same name, since all calls to the library or module  will include the namespace information.

In [8]:
# Grab all of the NumPy functions with namespace np
import numpy as np              

Next, we'll import matplotlib. This is another cell that will show up unchanges in nearly every Notebook that we do in *MCHE474*. The `%matplotlib inline` command tells the Notebook to include the plots inline with it, rather than plotting them in a seperate window. We import the `matplotlib.pyplot` module into the namepace `plt`. As you'll see, this means that nearly all of our plotting commands are prefixed by `plt.`.

In [9]:
%matplotlib inline

# Import the plotting functions 
import matplotlib.pyplot as plt

Finally we'll import the Control Systems Library. We don't specify a shorthand namespace, so we'll need to preface any commands from it with `control.`.

In [10]:
import control # This will import the control library.  

Now that we've imported the necessary libraries, let's walk through the basic use of them, roughly mirroring Section 2.9 of  [*Modern Control Systems (13th Edition)* by Richard Dorf and Robert Bishop](http://amzn.com/0134407628), the current (as of Fall 2017) textbook for *MCHE474*.

## Defining the Plant Transfer Function
We'll start by looking at the system in Figure 1. It's a simple mass-spring-damper system, with mass $m$ connected to ground through a spring $k$ and damper $c$. There is also a input force, $f$, acting on the mass.

<p style="text-align:center">
	<img src="http://crawlab.org/shared/MassSpringDamper_DirectForce_Horiz.png" alt="Example Mass-spring-damper System" width=50%><br>
    <strong>Figure 1: Example Mass-spring-damper System</strong>
</p>

The equation of  motion of this system is:

$ \quad m \ddot{x} + c \dot{x} + kx = f $

If we define $\omega_n$ to be the system's *natural frequency* and $\zeta$ as the system's *damping ratio*, we can rewrite this equation as:

$ \quad \ddot{x} + \frac{c}{m} \dot{x} + \frac{k}{m}x = \frac{f}{m} $

Then,

$ \quad \ddot{x} + 2\zeta\omega_n \dot{x} + \omega_n^2 x = \frac{f}{m} $

where

$ \quad \omega_n = \sqrt{\frac{k}{m}} $

and 

$ \quad 2\zeta\omega_n = \frac{c}{m} $

The transfer function of this system is:

$ \quad G(s) = \frac{X(s)}{F(s)} = \frac{1}{ms^2 + cs + k} $

Like the time-domain solution, we can also write this transfer function in terms of the natural frquency, $\omega_n$, and damping ratio, $\zeta$, of the system:

$ \quad G(s) = \frac{X(s)}{F(s)} = \frac{1/m}{s^2 + 2\zeta\omega_n s + \omega_n^2} $

Let's use this second form.  

We'll first define an array, which we'll call `num` that contains the numerator value. Since it only has one value, the command will be of the form:

    num = [1 / m]

To define the denominator, we need to create an array where the element represents the power of `s` for the constant there. In this case:

    den = [1, 2 * zeta * w_n, w_n**2]

To check this, work from the "right" side of the definition. `w_n**2` is $\omega_n^2$, which is mulitplied by $s^0$. The next element is `2 * zeta * w_n`, which is the $2\zeta\omega_n$ term and given its position in the first indexed element of the array is multiplied by $s^1$. This continues for all similar definitions.

***Note:*** This is one difference between Python and MATLAB. In Python, the commas are always needed between elements. In MATLAB, they are (sometimes) optional.

We'll first define the system parameters, picking the natural frequency and damping ratio to match the one used in the book:

$\quad \omega_n = \sqrt{2} $ rad/s

and

$ \quad \zeta = \frac{1}{2\sqrt{2}} $.

In [11]:
# Define the natural frequency. We use the numpy sqrt function, so we need to preface its call with np.
w_n = np.sqrt(2)    

# Define the damping ratio. 
zeta = 1 / (2 * np.sqrt(2))

# We'll also define the mass of the system
m = 1.0     # mass (kg)

In [12]:
# Define the numerator and denominator of the transfer function
num = [1 / m]
den = [1, 2 * zeta * w_n, w_n**2]

Now, that these are defined, we can pass them to the `tf` function of the library. Remember that since `tf` is a function in the `control` module, we need to use `control.tf()` to call it. We'll assign the variable `sys` to hold transfer function returned. 

In [13]:
# Define the transfer function form of the system defined by num and den
sys = control.tf(num, den)

We can check that the transfer function is defined correctly by printing `sys`.

In [14]:
print(sys)


     1
-----------
s^2 + s + 2



## Block Diagram Algebra

###  Series Connections
If two transfer functions are connected in series, with the output of one serving as the input to the other, as in the block diagram in Figure 2, we can use the [`series` function](http://python-control.readthedocs.io/en/latest/generated/control.series.html#control.series) in the Control System Toolbox to calculate the equivalent transfer function.

<p style="text-align:center">
	<img src="http://shared.crawlab.org/blockDiagram_seriesConnection.png" alt="Block Diagram of Series Connected Transfer Functions" width=50%><br>
    <strong>Figure 2: Block Diagram of Series Connected Transfer Functions</strong>
</p>

Let's consider the mass-spring-damper system as $G_2$ from this image and define $G_1$ to be:

$ \quad G_1(s) =  \frac{k_d s + k_p}{1} $

In [19]:
# First define kp and kd
kp = 1.0
kd = 0.1

# Define the G1 transfer function
g1_num = [kd, kp]
g1_den = [1]

g1_sys = control.tf(g1_num, g1_den)

Now, we can use the Control System Toolbox to calculate the transfer function of the simplified version of this block diagram, as shown in Figure 3.
<p style="text-align:center">
	<img src="http://shared.crawlab.org/blockDiagram_seriesConnection_Equiv.png" alt="Reduced Block Diagram of a Series Connection" width=35%><br>
    <strong>Figure 3: Reduced Block Diagram of a Series Connection</strong>
</p>

In [21]:
# Calculate the series connection
series_sys = control.series(g1_sys, sys)

# print the resulting transfer function
print(series_sys)


 0.1 s + 1
-----------
s^2 + s + 2



**Short aside on string formatting:**
We can nicely formatting our printing in Python. [This article](https://pyformat.info) contains a nice summary of the `.format()` method of doing so that our notebooks will tpyically use. There is also a new (in Python 3.6) [`f-string` formatting](https://docs.python.org/3.6/reference/lexical_analysis.html#f-strings) method that may be more convenient. So, we can print the transfer function above a bit more nicely by either:

In [31]:
print('The resulting transfer function is: {}'.format(series_sys))

The resulting transfer function is: 
 0.1 s + 1
-----------
s^2 + s + 2



or:

In [30]:
print(f'The resulting transfer function is: {series_sys}')

SyntaxError: invalid syntax (<ipython-input-30-7355961d687e>, line 1)

### Unity Feedback Connections
If we close the loop around the $G_1$ and $G_2$, as shown in Figure 4, we can use the [`feedback` function](http://python-control.readthedocs.io/en/latest/generated/control.feedback.html#control.feedback) of the library to calculate the resulting closed-loop transfer function.
<p style="text-align:center">
	<img src="http://shared.crawlab.org/blockDiagram_seriesConnection_feedback.png" alt="Block Diagram with Feedback" width=50%><br>
    <strong>Figure 4: Block Diagram with Feedback</strong>
</p>

The feedback function will also work if there is a block in the feedback path. By default, the function assumes that the system has unity feedback, as this one does. The feedback is also is negative (as is most often the case), which is the default for the function. We should specify positive, if we needed.

In [34]:
# Calculate the closed-loop transfer function
sys_closedLoop = control.feedback(series_sys)

# Then print out the result
print('The closed-loop transfer function is {}'.format(sys_closedLoop))

The closed-loop transfer function is 
   0.1 s + 1
---------------
s^2 + 1.1 s + 3



This matches the reduced form of that block diagram:

$\quad G_{CL} = \frac{G_1 G_2}{1 + G_1 G_2} $

### Non-unity Feedback Connections
If there is a block in the feedback path, like that shown in Figure 5, then the system no longer has unity feedback. In this case, we need to include that transfer function in the call to `feedback`. Below, we'll first define that transfer function, $H(s)$, then include it in the function call.

<p style="text-align:center">
	<img src="http://shared.crawlab.org/blockDiagram_seriesConnection_nonUnityFeedback.png" alt="Block Diagram with Non-unity Feedback" width=50%><br>
    <strong>Figure 5: Block Diagram with Non-unity Feedback</strong>
</p>

In [36]:
# First define the numerator and denominator of H(s)
h_num = [1]
h_den = [0.05, 1]

# Now, define the transfer function for H(s)
h_sys = control.tf(h_num, h_den)

Once we've define the transfer function of the block in the feedback loop, we can include it in the call to the `feedback` function of the library:

In [37]:
sys_closedLoop_withH = control.feedback(series_sys, h_sys)

and print out the result:

In [39]:
print('The closed-loop transfer function for the non-unity feedback system is {}'.format(sys_closedLoop_withH))

The closed-loop transfer function for the non-unity feedback system is 
    0.005 s^2 + 0.15 s + 1
-------------------------------
0.05 s^3 + 1.05 s^2 + 1.2 s + 3



### Parallel Connections
Parallel connections, like the one shown in Figure 5, are handled by, you guessed it, the [`parallel` function](http://python-control.readthedocs.io/en/latest/generated/control.parallel.html#control.parallel) of the library.

<p style="text-align:center">
	<img src="http://shared.crawlab.org/blockDiagram_parallelConnection.png" alt="Block Diagram with a Parallel Connection" width=50%><br>
    <strong>Figure 5: Block Diagram with a Parallel Connection</strong>
</p>
We'll use the previously defined transfer functions for this example.

In [41]:
# Calculate the transfer function
sys_parallel = control.parallel(sys, g1_sys)

# Then, print it out
print('The transfer function for the parallel connection is {}'.format(sys_parallel))

The transfer function for the parallel connection is 
0.1 s^3 + 1.1 s^2 + 1.2 s + 3
-----------------------------
         s^2 + s + 2



For these parallel connections, we know that the reduced block diagram results in a transfer function of:

$ \quad G_{parallel}(s) = G_1 + G_2 $

Because of this, we just add the two transfer functions and should get the same result as the `parallel` function:

In [44]:
sys + g1_sys


0.1 s^3 + 1.1 s^2 + 1.2 s + 3
-----------------------------
         s^2 + s + 2

We can, of course, apply these functions multiple times (similar to how we did for the feedback loop examples above) to achieve the level of block diagram reduction we desire. We'll do so for some examples in other notebooks.

<hr>


#### Licenses
Code is licensed under a 3-clause BSD style license. See the licenses/LICENSE.md file.

Other content is provided under a [Creative Commons Attribution-NonCommercial 4.0 International License](http://creativecommons.org/licenses/by-nc/4.0/), CC-BY-NC 4.0.

In [15]:
# Ignore this cell - We just update the CSS to make the notebook look a little bit better and easier to read

# Improve the notebook styling -- Run this first
import codecs
from IPython.core.display import HTML
css_file = 'styling/CRAWLAB_IPythonNotebook.css'
HTML(codecs.open(css_file, 'r', 'utf-8').read())